In [1]:
import numpy as np
from matplotlib import pyplot as plt
from NeuralNetwork import NeuralNetwork
from functions import *
from Layer import Layer, Input
from utils import *
import pickle

In [2]:
df = get_data('/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/MONK/monks-1.train')
df_test = get_data('/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/MONK/monks-1.test')

In [3]:
#df = get_data(r"\Users\s512fj-ej021t\OneDrive\Desktop\ML\monks-1.train")
#df_test = get_data(r"\Users\s512fj-ej021t\OneDrive\Desktop\ML\monks-1.test")

In [4]:
X_train, y_train = df.drop(columns=['target','id']).to_numpy().T, df['target'].apply(lambda x: int(x)).to_numpy().T
X_test, y_test = df_test.drop(columns=['target','id']).to_numpy().T, df_test['target'].apply(lambda x: int(x)).to_numpy().T

In [5]:
X_train = onehot_encoding(X_train)

In [6]:
params = {
          'eta' : [0.005,0.01,0.05,0.1,0.5],
          'lam' : [0.0, 0.01,0.03],
          'alpha':[0.1,0.5,0.9],
          'epochs': [500],
          'n_batch' : [1, 31, 'batch'],
          'scale_eta_batchsize' : ['sqrt','lin',None], #'sqrt' per eta * sqrt(n_batch), 'lin' per eta * n_batch
          
          'dim_hidden' : [2,3,4],
          'hidden_act_func' : ['relu','tanh']
        }


In [11]:
params = {
          'eta' : [0.005,0.01,0.5],
          'lam' : [0.0, 0.01],
          'alpha':[0.5,0.9],
          'epochs': [500],
          'n_batch' : ['batch'],
          'scale_eta_batchsize' : [None], #'sqrt' per eta * sqrt(n_batch), 'lin' per eta * n_batch
          
          'dim_hidden' : [2,3],
          'hidden_act_func' : ['relu'],

          'loss' : ['binary_crossentropy'],
          'output_act_func' : ['sigm']
        }


In [8]:
early_stopping = {'patience' : 150,
                  'monitor' : 'val_accuracy',
                  'verbose' : 0,
                  'compare_function': np.greater_equal}

reduce_eta = {'patience' : 75,
              'monitor' : 'val_accuracy',
              'factor' : 0.5,
              'verbose' : 0,
              'compare_function': np.greater_equal}

callbacks = {'early_stopping': None,
             'reduce_eta': None}

In [12]:
best_comb, param_grid = grid_search(X_train, y_train.reshape((1,X_train.shape[1])), params, 5, [accuracy,MSE], callbacks)
166

Starting grid_search...
Grid of parameters: {'eta': [0.005, 0.01, 0.5], 'lam': [0.0, 0.01], 'alpha': [0.5, 0.9], 'epochs': [500], 'n_batch': ['batch'], 'scale_eta_batchsize': [None], 'dim_hidden': [2, 3], 'hidden_act_func': ['relu'], 'loss': ['binary_crossentropy'], 'output_act_func': ['sigm']}
-------------------------------------------------
Starting params 1/24: {'eta': 0.005, 'lam': 0.0, 'alpha': 0.5, 'epochs': 500, 'n_batch': 'batch', 'scale_eta_batchsize': None, 'dim_hidden': 2, 'hidden_act_func': 'relu', 'loss': 'binary_crossentropy', 'output_act_func': 'sigm'}
Results:
train_loss_mean: 2.07e-01
train_loss_std: 6.28e-02
val_loss_mean: 4.16e-01
val_loss_std: 1.55e-01
train_accuracy_mean: 9.25e+01
train_accuracy_std: 3.76e+00
val_accuracy_mean: 8.30e+01
val_accuracy_std: 4.79e+00
train_MSE_mean: 5.83e-02
train_MSE_std: 2.40e-02
val_MSE_mean: 1.25e-01
val_MSE_std: 4.06e-02
Elapsed time: 7.086872 s
-------------------------------------------------
Starting params 2/24: {'eta': 0.005

KeyboardInterrupt: 

In [ ]:
best_comb

{'eta': 0.01,
 'lam': 0.0,
 'alpha': 0.5,
 'epochs': 500,
 'n_batch': 1,
 'scale_eta_batchsize': None,
 'dim_hidden': 3,
 'hidden_act_func': 'relu',
 'results': {'train_loss_mean': 0.12294714136838696,
  'train_loss_std': 0.0506226861970321,
  'val_loss_mean': 0.20320056148828286,
  'val_loss_std': 0.09086906646796203,
  'train_accuracy_mean': 97.98989898989899,
  'train_accuracy_std': 2.4619127080841645,
  'val_accuracy_mean': 97.6,
  'val_accuracy_std': 4.800000000000001,
  'train_MSE_metric_mean': 0.015107486359720673,
  'train_MSE_metric_std': 0.01868979067643282,
  'val_MSE_metric_mean': 0.022464635160999117,
  'val_MSE_metric_std': 0.036266188039089015},
 'elapsed_time': 108.96439099311829}

In [ ]:
def save_dict_to_file(dictionary, filename):
    """
    Save a dictionary to a file using pickle.

    Parameters:
    - dictionary: The dictionary to be saved.
    - filename: The name of the file to save the dictionary to.
    """
    with open(filename, 'wb') as file:
        pickle.dump(dictionary, file)

In [ ]:
save_dict_to_file(best_comb,'/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/output/best_comb2.pkl')
save_dict_to_file(param_grid,'/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/output/param_grid2.pkl')